<a href="https://colab.research.google.com/github/ultra151/cv/blob/main/20_KeyPoints%ED%83%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install torch torchvision

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 22.8 MB/s eta 0:00:00


In [ ]:
%cd /content/gdrive/MyDrive/pytest_img/YOLO/

/content/gdrive/MyDrive/pytest_img/YOLO


In [ ]:
!ls

data.yaml    nets		  README.roboflow.gdoc	sample.mp4  train  walking.mp4	yolov5s.pt
gangnam.jpg  README.dataset.gdoc  results		test	    valid  yolov5


In [ ]:
from ultralytics import YOLO
model = YOLO("../models/yolov8m-pose.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 50.8M/50.8M [00:00<00:00, 156MB/s]


In [ ]:
seconds = 1

videl_path = '/content/gdrive/MyDrive/pytest_img/YOLO/walking.mp4'
output_dir = '/content/gdrive/MyDrive/pytest_img/YOLO/results'

In [ ]:
import torch
def predict(frame, iou=0.7, conf=0.25):
    results = model(
        source = frame,
        device = "0" if torch.cuda.is_available() else "cpu",
        iou = iou,
        conf = conf,
        verbose = False,
    )
    return results[0]

In [ ]:
import cv2
import numpy as np

def draw_keypoints(result, frame):
    connections = [
        ([4,2,0,1,3], (0,255,0)),
        ([10,8,6,5,7,9], (255,0,0)),
        ([6,12,11,5], (255,0,255)),
        ([12,14,16], (0,165,255)),
        ([11,13,15], (0,165,255))
    ]
    for kps in result.keypoints:
        kps = kps.data.squeeze()
        nkps = kps.cpu().numpy()

        for group, color in connections:
            for i in range(len(group) - 1):
                idx1, idx2 = group[i], group[i+1]
                x1,y1,score1 = nkps[idx1]
                x2,y2,score2 = nkps[idx2]

                if score1 > 0.5 and score2 > 0.5:
                    point1 = (int(x1), int(y1))
                    point2 = (int(x2), int(y2))

                    cv2.circle(frame, point1, 3, (0,0,255), cv2.FILLED)
                    cv2.putText(frame, str(idx1), point1, cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)

                    cv2.circle(frame, point2, 3, (0,0,255), cv2.FILLED)
                    cv2.putText(frame, str(idx2), point2, cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,255), 1)
                    cv2.line(frame, point1, point2, color, 1)
    return frame



In [ ]:
import cv2

capture = cv2.VideoCapture(videl_path)
if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

fps = capture.get(cv2.CAP_PROP_FPS)
total_frames = int(fps * seconds)
print('total 프레임:', total_frames)

total 프레임: 25


In [ ]:
import os

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

frame_count = 0
while frame_count < total_frames:
    ok, frame = capture.read()
    if not ok:
        print("프레임 읽기에 실패했습니다. 종료.")
        break

    result = predict(frame)
    results = draw_keypoints(result, frame)

    output_path = os.path.join(output_dir, f"output_{frame_count:04d}.jpg")
    cv2.imwrite(output_path, results)
    print("Saved to:", output_path)

    frame_count += 1
    #key = cv2.waitKey(1)
    #if key == ord('q'):
    #    print("사용자가 종료를 요청했습니다.")
    #    break

#capture.release()
#cv2.destroyAllWindows()

Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0000.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0001.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0002.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0003.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0004.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0005.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0006.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0007.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0008.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0009.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0010.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0011.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/output_0012.jpg
Saved to: /content/gdrive/MyDrive/pyte